##### Seq2Seq Language Translation model
##### Dataset used : French Language (fra.text)


Build a Seq2Seq Language Translation model for any language pair of your choice. (See Datasets: http://www.manythings.org/anki/ )

(1) Build a character-level model 

(2) Build a word level model

(3) Build a word model with attention

### Q1. Character level model

In [110]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.python.keras.models import Model


#### Get the inputs and encode them

In [111]:
sample_len= 10000
df = 'fra.txt'
input_texts = []
target_texts = []
ip_chars = set()
op_chars = set()
with open(df, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
    
#Inputs have two parts, the English text ie input_text and input target ie target_text    
for line in lines[: min(sample_len, len(lines) - 1)]:
    input_text, target_text = line.split('\t')[:2]
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in ip_chars:
            ip_chars.add(char)
    for char in target_text:
        if char not in op_chars:
            op_chars.add(char)

ip_chars = sorted(list(ip_chars))
op_chars = sorted(list(op_chars))
num_encoder_tokens = len(ip_chars)
num_decoder_tokens = len(op_chars)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


In [112]:
input_texts[:10]

['Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Who?',
 'Wow!',
 'Fire!',
 'Help!',
 'Jump.']

In [113]:
target_texts[:10]

['\tVa !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tSaute.\n']

In [114]:
ip_tokens = dict([(char, i) for i, char in enumerate(ip_chars)])
op_tokens = dict([(char, i) for i, char in enumerate(op_chars)])
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, ip_tokens[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, op_tokens[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, op_tokens[char]] = 1.

### Define the character-level model 

In [115]:
batch_size = 128 
epochs = 100
latent_dim = 256 

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [116]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])
model.summary()

Model: "functional_31"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           [(None, None, 93)]   0                                            
__________________________________________________________________________________________________
lstm_10 (LSTM)                  [(None, 256), (None, 335872      input_21[0][0]                   
__________________________________________________________________________________________________
lstm_11 (LSTM)                  [(None, None, 256),  358400      input_22[0][0]                   
                                                                 lstm_10[0][1]        

In [117]:
history1 = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.3)
model.save('s2s.h6')

Epoch 1/100
55/55 [==============================] - 100s 2s/step - loss: 1.0271 - acc: 0.0478 - val_loss: 1.0175 - val_acc: 0.0631
Epoch 2/100
55/55 [==============================] - 124s 2s/step - loss: 0.8766 - acc: 0.0656 - val_loss: 0.9705 - val_acc: 0.0718
Epoch 3/100
55/55 [==============================] - 141s 3s/step - loss: 0.8076 - acc: 0.0888 - val_loss: 0.8632 - val_acc: 0.1146
Epoch 4/100
55/55 [==============================] - 141s 3s/step - loss: 0.7131 - acc: 0.1159 - val_loss: 0.7794 - val_acc: 0.1333
Epoch 5/100
55/55 [==============================] - 143s 3s/step - loss: 0.6422 - acc: 0.1324 - val_loss: 0.7214 - val_acc: 0.1450
Epoch 6/100
55/55 [==============================] - 144s 3s/step - loss: 0.5960 - acc: 0.1419 - val_loss: 0.6801 - val_acc: 0.1537
Epoch 7/100
55/55 [==============================] - 146s 3s/step - loss: 0.5643 - acc: 0.1491 - val_loss: 0.6522 - val_acc: 0.1613
Epoch 8/100
55/55 [==============================] - 147s 3s/step - loss: 0.

In [125]:
print("Validation Accuracy:", max(history1.history['val_acc']))
print("Training Accuracy:", max(history1.history['acc']))

Validation Accuracy: 0.223548024892807
Training Accuracy: 0.2826876640319824


### Inference Mode

In [118]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in ip_tokens.items())
reverse_target_char_index = dict((i, char) for char, i in op_tokens.items())

In [119]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, op_tokens['\t']] = 1.
    #target_seq[0, 0, op_tokens[' ']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]

    return decoded_sentence

In [120]:
 for seq_index in range(30):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    number = str(seq_index)
    print('Sentence Number: '+ number)
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Sentence Number: 0
Input sentence: Go.
Decoded sentence: Appele !

Sentence Number: 1
Input sentence: Hi.
Decoded sentence: Salut !

Sentence Number: 2
Input sentence: Hi.
Decoded sentence: Salut !

Sentence Number: 3
Input sentence: Run!
Decoded sentence: Courez !

Sentence Number: 4
Input sentence: Run!
Decoded sentence: Courez !

Sentence Number: 5
Input sentence: Who?
Decoded sentence: Qui l'a comment !

Sentence Number: 6
Input sentence: Wow!
Decoded sentence: Attaquoi !

Sentence Number: 7
Input sentence: Fire!
Decoded sentence: Au feu !

Sentence Number: 8
Input sentence: Help!
Decoded sentence: À l'aide !

Sentence Number: 9
Input sentence: Jump.
Decoded sentence: Saute.

Sentence Number: 10
Input sentence: Stop!
Decoded sentence: Arrête-toi !

Sentence Number: 11
Input sentence: Stop!
Decoded sentence: Arrête-toi !

Sentence Number: 12
Input sentence: Stop!
Decoded sentence: Arrête-toi !

Sentence Number: 13
Input sentence: Wait!
Decoded sentence: Attendez !

Sentence Number: 

### Observations :
1. The Seq2seq Model built using the Character-based model gives a Validation Accuracy of 22%.
* The model predicts most of the single words correctly.
* However, the model seems to struggle with successive words in small sentences.
2. This is not a very acceptable Model Accuracy.
3. In the next part, we will build the Seq2seq model using word-based tokenization and validate the overall validation data accuracy and compare the two models.
